# Assignment 3 option 2: ConvNet
In this assignment you will train a ConvNet to predict the language of a
surname from its spelling.

In [0]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=f24b73325d2f124b52003c4ce0e2333d016fbe020bd4dd7cb2d33ee3c2c6abbc
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import random
from google.colab import drive
import matplotlib.pyplot as plt
from scipy import sparse
from collections import Counter
from operator import itemgetter
from random import sample 

time: 349 ms


# Component 1: Preparing Data
Read in the training data, determine the maximum
length of all the names in the dataset, determine the number of unique
characters in the text and set up mapping functions - one mapping
each character to a unique index and another mapping each index to a character.

## 0.1 Read in the data and get it ready
read in acsii_names.txt
put all names and ther labels un a call array all_names and a vector y 
convert all uppercase letters to lowercase

In [0]:
# upload txt file to colab: https://buomsoo-kim.github.io/colab/2018/04/15/Colab-Importing-CSV-and-JSON-files-in-Google-Colab.md/
from google.colab import files
uploaded = files.upload()

Saving ascii_names.txt to ascii_names.txt
Saving Validation_Inds.txt to Validation_Inds.txt
time: 17 s


In [0]:
# preporocess data
names_train = []
names_val = []
labels_train = []
labels_val = []

name_data = "ascii_names.txt"
validation_data = "Validation_Inds.txt"
names = uploaded[name_data].decode("utf-8").split("\n")
validation_idx = uploaded[validation_data].decode("utf-8").split()
names = list(filter(lambda x: x != "", names))
for i in range(len(names)):
  line = names[i].split(",")[0].rsplit(' ', 1)
  if len(line)>1 and line[1] != '':
    if str(i+1) in validation_idx: 
      names_val.append(line[0].lower())
      labels_val.append(int(line[1]))
    else:
      names_train.append(line[0].lower())
      labels_train.append(int(line[1]))

print(len(names_val), len(names_train), len(labels_val), len(labels_train))

252 19798 252 19798
time: 137 ms


In [0]:
C = sorted(set([i for ele in names_train for i in ele])) # set of unique characters in names
d = len(C) # number of unique characters in names
n_len =  len(max(names_train, key=len)) # size of longest name in names
K = len(set(labels_train)) # number of classes
char_to_ind =  { val : id for id,val in enumerate(C) } # dict with char and index in set C
print(n_len)

19
time: 44.5 ms


In [0]:
def createOneHot_name(_name, _char_to_ind, _n_len, _d):
  values =  [_char_to_ind[val] for val in list(_name)]
  one_hot = np.eye(_d)[values].copy()
  one_hot.resize((_n_len, _d),  refcheck=False)
  return one_hot.T 
createOneHot_name('abc', char_to_ind, 4, 6).flatten('F')

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0.])

time: 7.72 ms


In [0]:
# create X as one hot encoding for names: columns represent names, rows the vectorized information
def createOneHot_X(names):
  X = np.zeros((d*n_len, len(names)))
  for id,name in enumerate(names):
    X[:,id] = createOneHot_name(name, char_to_ind, n_len, d).flatten('F')
  return X

print(createOneHot_X(names_train).shape, len(names_train))
print(createOneHot_X(names_val).shape, len(names_val))

(532, 19798) 19798
(532, 252) 252
time: 395 ms


In [0]:
# create one hot encoding for labels --> each column corresponds to one name
def createOneHot_Y(labels):
  onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
  Y = onehot_encoder.fit_transform(np.array(labels).reshape(-1,1)).astype(int).T
  return Y
print(createOneHot_Y(labels_train).shape)
print(createOneHot_Y(labels_val).shape)

(18, 19798)
(18, 252)
time: 20.5 ms


In [0]:
# plot all graphs
def plotResults(costs_train, accs_train, costs_validate, accs_validate, safeto, save=False):
    # plotting costs
    plt.plot(costs_train, label = "training")
    plt.plot(costs_validate, label = "validation")
    plt.xlabel('saving steps')
    plt.ylabel('cost')
    plt.suptitle('Cost')
    plt.legend()
    if save:
        plt.savefig('Result_Pics/'+safeto+'cost.png')
    plt.show()  
    
    # plotting accuracy
    plt.plot(accs_train, label = 'training')
    plt.plot(accs_validate, label = 'validation')
    plt.xlabel('saving steps')
    plt.ylabel('accuracy')
    plt.suptitle('Accuracy')
    plt.legend()
    if save:
        plt.savefig('Result_Pics/'+safeto+'acc.png')
    plt.show()

time: 7.57 ms


# 2 Convolutional Network



In [0]:
# object for parameters 
random.seed( 30 )

class ConvNet():
  def __init__(self, n_classes, eta, rho, ns, ks, d, n_len, K, L):
    self.L = L # number of conv layers
    self.n = ns # filters applied at layer  []
    self.k = ks # the width of the filters applied at layer []
    self.n_len = n_len
    self.d = d
    self.F = {}
    self.eta = eta
    self.rho = rho
    self.M_filter = {}
    self.len_f = [] # lens used for calculation MF matrix for each layer []
    self.K = K
    self.confusion = []

    self.len_f.append(n_len)
    for i in range(L): # for each layer
      # n len f
      n_len_new =  self.len_f[-1]-ks[i]+1
      self.len_f.append(n_len_new)
      # filters
      if i == 0:
        self.F[i] = np.zeros((self.d, self.k[i], self.n[i]) ) 
      else:
        self.F[i] = np.zeros((self.n[i-1], self.k[i], self.n[i]) )
    self.W = np.zeros((n_classes, self.len_f[-1] * self.n[-1]))  # Kn2nlen2

  def init(self,mu, sigma):
    # init filters
    for i in range(self.L): # for every layer
        self.F[i] = np.random.normal(mu, sigma, self.F[i].shape ) 
    # init W
    self.W = np.random.normal(mu, sigma, self.W.shape) 

  def makeMFMatrix(self, F, n_len): # called for a specific layer - nothing to change
    (d, k, nf) = F.shape
    M_filter = np.zeros(((n_len -k+1)*nf, n_len*d))
    Vec_filter = F.reshape((d*k, nf), order = 'F').T
    #print (F.shape, Vec_filter.shape, M_filter.shape)

    for i in range(n_len - k + 1):
      row_start = i * nf
      row_end = (i + 1) * nf
      col_start = d * i
      col_end = d * i + d * k      
      M_filter [row_start:row_end,col_start:col_end] = Vec_filter
    
    return M_filter

  def makeMXMatrix(self, x_input, d, k, nf, stride = 1):    #d,k,nf = size(F) # not used in improves version - nothing to change
    n_len = int(len(x_input)/d)
    M_input = np.zeros((nf*(n_len -k+1),nf*d*k))
    x_input = x_input.reshape((d, n_len), order='F') # todo: try to get a version without all the shaping
    
    for i in range((n_len -k+1)):
      row_start = i * nf   
      vec =  (x_input[:, i:k+i].reshape((d*k, 1), order='F')).T 
      #vec = x_input[i*d*stride: k*d*stride+i*d*stride]
      for j in range(nf):     
        M_input [row_start+j, j*d*k: (j+1)*d*k] = vec
  
    return M_input

  ''' np.kron is very slow in this loop. Do not use it! Use makeMXMatrix instead ''' # not used in improves version - nothing to change
  def makeMXMatrixKron(self, x_input, d, k, nf, stride = 1):    #F.shape = d x k x nf  , d = 28, k&nf to decide
    n_len = int(len(x_input)/d)               	    # X.shape = d x n_len
    M_input = np.zeros((nf*(n_len -k+1),nf*d*k))    # size of MX
    vec = np.zeros(((n_len -k+1),d*k))              # one vector square

    x_input = x_input.reshape((d, n_len), order='F') # todo: try to get a version without all the shaping
    print(x_input.shape, d, k, nf, n_len, (n_len -k+1), M_input.shape )

    for i in range((n_len -k+1)):
      vec[i] =  (x_input[:, i:k+i].reshape((d*k, 1), order='F')).T 
      #vec[i]= x_input[i*d*stride: k*d*stride+i*d*stride]
      if i == 0:
        M_input =  np.kron(np.eye(nf),vec[i])
      else:
        M_input = np.vstack((M_input, np.kron(np.eye(nf),vec[i])))
  
    return M_input

  def makeMXMatrixVec(self, x_input, d, k):   
    n_len = int(len(x_input)/d)
    vec = np.zeros(((n_len -k+1),d*k))
    x_input = x_input.reshape((d, n_len), order='F') 
    
    for i in range((n_len -k+1)):
      vec[i] =  (x_input[:, i:k+i].reshape((d*k, 1), order='F')).T 
  
    return vec

  def softmax(self, scores): # not layer specific - nothing to change
    f = np.exp(scores - np.max(scores))  # avoiding nan for large numbers
    return f / f.sum(axis=0)

  def evaluateClassifier(self, X_batch, MFs, W): # forward pass
    X_batches = [X_batch]
    for i in range(len(MFs)): # iterate over MFs - one MF entry for each layer 
      new_X_batch = np.maximum(np.dot(MFs[i],X_batches[i]), 0)
      X_batches.append(new_X_batch)

    # fully connected
    S_batch = np.dot(W ,X_batches[-1])
    # Apply the SoftMax operator to each column
    P_batch = self.softmax(S_batch)

    return X_batches, P_batch # return list of x_batches instead of 1 and 2 - keep in mind [0] is initial xBatch, [1] x_batch1, [i] x_batchi

  def crossEntropyLoss(self, X, Y, MFs, W):
    _, P = self.evaluateClassifier(X, MFs, W)
    Y = np.reshape(Y, (P.shape))
    loss = -(np.multiply(Y, np.log(P)))  # check this:  - np.sum(Y*np.log(P)), removed  np.sum
    return loss

  def cost(self, X, Y, MFs, W):
    J = np.sum(self.crossEntropyLoss(X, Y, MFs, W)) / X.shape[1]
    return J

  def computeAccuracy(self, X, labels, MFs, W):
    _, P = self.evaluateClassifier(X, MFs, W)
    pred = np.argmax(P, axis=0)
    pred = [x + 1 for x in pred]
    #print(pred)
    acc = np.count_nonzero(np.array(pred) == np.array(labels)) / X[1].size
    return acc

  # old: def passBackward(self, X_batch, X_batch1, X_batch2, P_batch, Y_batch, MFs, MX_prec):
  def passBackward(self, X_batches, P_batch, Y_batch, MFs, MX_prec):
    # assume forward pass / evaluate classifier is done before
    grad_W = np.zeros(self.W.shape)
    grad_F = []
    for i in range(len(self.F)): # layer specific
      grad_F.append(np.zeros(self.F[i].shape))
    
    # fully conected layer
    G_batch = - (Y_batch - P_batch)
    grad_W = np.dot(G_batch, X_batches[-1].T) / X_batches[-1].shape[1] # use last one in list

    '''generalize from here'''

    # for each layer going backwards
    for i in range(len(self.F)-1, -1, -1):
      # Propagate the gradient to the previous layer through the fully conected /convolutional layer and ReLu operation:
      if i == (len(self.F)-1): # fully conected - can be changes to make it more general
        G_batch = np.dot(self.W.T, G_batch)
      else:
        G_batch = np.dot(MFs[i+1].T, G_batch) # MF from previous layer
      
      G_batch = np.multiply(G_batch, np.where(X_batches[i+1] >0, 1,0)) # from previous

      # compute gradients w.r.t the layer
      for j in range(X_batches[i].shape[1]): # todo
        gj = G_batch[:,j]
        xj = X_batches[i][:,j] # todo

        # improvement 2
        if i == 0: 
          M_inputj_vec = self.makeMXMatrixVec(xj, self.d, self.k[i]) # input dimension for first layer is d
        else: 
          M_inputj_vec = self.makeMXMatrixVec(xj, self.n[i-1], self.k[i] ) # input dimension for other layers if previous n

        leni = int( gj.shape[0] / self.n[i]) # first dim of g_new         # todo??? review this n
        g_efficient = gj.reshape(leni, self.n[i])    # todo??? review this n
        v2 = np.dot(M_inputj_vec.T, g_efficient) 
        grad_F[i] += v2.reshape(grad_F[i].shape, order='F') / X_batches[i].shape[1]     # todo grad_F and x_batch

    return grad_W, grad_F
    
  def num_grads(self, X, Y, MF, W, h):
    '''Translation from Matlab template provided by classmate (Many thanks to Filip)'''
    dW = np.zeros_like(W)
    (a, b) = W.shape
    for i in range(a):
        for j in range(b):
            C = []
            
            for m in [-1, 1]:
                W_try = np.copy(W)
                W_try[i, j] += m * h
                C.append(self.cost(X, Y, MF, W_try))
            dW[i, j] = (C[1] - C[0]) / (2 * h)

    dF = [np.zeros(self.F[f].shape) for f in self.F]
    for i in range(len(self.F)):
        (a, b, c) = self.F[i].shape
        for j in range(a):
            for k in range(b):
                for q in range(c):
                    C = []

                    for m in [-1, 1]:
                        Fi_try = np.copy(self.F[i])
                        Fi_try[j, k, q] += m * h
                        MFi_try = self.makeMFMatrix(Fi_try, self.len_f[i])
                        MF_lst = []
                        for ii in range(len(self.F)):
                          MF_lst.append(MFi_try if ii == i else MF[ii])
                        C.append(self.cost(X, Y, MF_lst, W))
                    dF[i][j, k, q] = (C[1] - C[0]) / (2 * h)
    return dW, dF
    
  def computeRelativeError(self, ga ,gn ,eps):
    return np.absolute(np.subtract(ga, gn))/np.maximum(np.add(np.absolute(ga), np.absolute(gn)), np.full(ga.shape, eps))
  
  def generateMiniBatches(self, n_batch, X, Y):
    n = X[1].size
    X_batches = []
    Y_batches = []
    batch_index = []

    for j in range(int(n / n_batch)):
      j_start = j * n_batch
      j_end = (j + 1) * n_batch

      X_batch = X[:, j_start:j_end]
      Y_batch = Y[:, j_start:j_end]

      X_batches.append(X_batch)
      Y_batches.append(Y_batch)
      batch_index.append(( j_start,j_end))

    return X_batches, Y_batches, batch_index

  def miniBatchGD(self, X, Y,  X_val, Y_val, labels_train, labels_val, n_batch, n_update, n_epochs, balanced):
    costs_train = []
    accs_train = []
    costs_val = []
    accs_val = []
    MFs =[]
    t = 0
    #F = self.F.copy()
    v_W = np.zeros(self.W.shape)
    v_F = []
   
    # get indexes
    index_dict = self.create_index_dict(labels_train)
    
    # construct M_filters
    for i in range(len(self.F)):
      MFs.append(self.makeMFMatrix(self.F[i], self.len_f[i]))
      v_F.append(np.zeros(self.F[i].shape))

     # precompute MX for the first layer for each batch  
     # improvement 1 - not efficient
    MX_prec = []
    '''
    for j in range(X.shape[1]):
      xj = X[:,j]
      MX_prec.append(sparse.csr_matrix(self.makeMXMatrix(xj, self.d, self.k[0], self.n[0])) ) 

    print("precomputing mx")
    MX_prec = [sparse.csr_matrix(self.makeMXMatrix(X[:,x], self.d, self.k[0], self.n[0]))  for x in range(X.shape[1])]
    print("precomputing mx done ")'''

    for epoch in range(n_epochs):  # iterate over epoch
      #print("epoch", epoch)

      if balanced:
        # get balanced data
        X_balanced, y_balanced = self.createBalancedDataset(X, labels_train, index_dict, self.K)
        #Y_balanced = onehot_encoder.fit_transform(np.array(y_balanced).reshape(-1,1)).astype(int).T
        Y_balanced = createOneHot_Y(y_balanced)

        # construct batches from balanced data
        X_batches, Y_batches, batch_index = self.generateMiniBatches(n_batch, X_balanced, Y_balanced)
      else: 
        # construct batches from unbalanced data
        X_batches, Y_batches, batch_index = self.generateMiniBatches(n_batch, X, Y)

      # construct MF prec for batches
      # improvement 1 - not efficient
      '''
      MX_prec_batches = []
      for i in range(len(X_batches)):
        MX_prec_batches.append(MX_prec[batch_index[i][0]:batch_index[i][1]] ) '''

      #MX_prec_batches = [MX_prec[batch_index[i][0]:batch_index[i][1]] for i in range(len(X_batches))]
      
      for idx, X_batch in enumerate(X_batches, start=0):  # iterate over mini-batches
        Y_batch = Y_batches[idx]

         #save acc and cost every n_update step
        if t % (n_update-1) == 0:
          costs_train.append(self.cost(X, Y, MFs, self.W))
          accs_train.append(self.computeAccuracy(X, labels_train, MFs, self.W))
          costs_val.append(self.cost(X_val, Y_val, MFs, self.W))
          accs_val.append(self.computeAccuracy(X_val, labels_val, MFs, self.W))
        
        # pass forward
        X_batches_fw, P_batch = self.evaluateClassifier(X_batch, MFs, self.W)

        # compute gradients / pass backwards

        #MX_prec_batch = MX_prec[batch_index[idx][0]:batch_index[idx][1]]
        grad_W, grads_F = self.passBackward(X_batches_fw, P_batch, Y_batch, MFs,[])
        #grad_W, grads_F = self.passBackward(X_batch, X_batch1, X_batch2, P_batch, Y_batch, MFs, MX_prec_batch) # improvement 1 - not efficient
        
        # momentum 
        # adjust learnung parameters 
        v_W = v_W * self.rho - self.eta * grad_W
        self.W += v_W

        # compute MFs
        MFs = []
        for i in range(len(self.F)): # layer specific       
          v_F[i] = v_F[i] * self.rho - self.eta * grads_F[i]
          self.F[i] += v_F[i]
          MFs.append(self.makeMFMatrix(self.F[i], self.len_f[i]))
  
        # increase t
        t += 1
    
    costs_train.append(self.cost(X, Y, MFs, self.W))
    accs_train.append(self.computeAccuracy(X, labels_train, MFs, self.W))
    costs_val.append(self.cost(X_val, Y_val, MFs, self.W))
    accs_val.append(self.computeAccuracy(X_val, labels_val, MFs, self.W))

    #print confusion matrix for train and validate
    self.printConfusion(X, MFs, self.W, labels_train )
    self.printConfusion(X_val, MFs, self.W, labels_val )
 
    return [costs_train, accs_train], [costs_val, accs_val], self.W, self.F

  def printConfusion(self, X, MFs, W, y ):
    _, P = self.evaluateClassifier(X, MFs, W)
    pred = np.argmax(P, axis=0)
    pred = [x + 1 for x in pred]
    # how to construct confusion matrix: https://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python
    y_actu = pd.Series(y, name='Actual')
    y_pred = pd.Series(pred, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred,  margins=False)
    self.confusion.append(df_confusion)
    #display(df_confusion)

  def create_index_dict(self,  y_unbalanced):
    # filter the indexes
    indexes_for_classes = {} 
    for i, value in enumerate(y_unbalanced):
      if value not in indexes_for_classes:
        indexes_for_classes[value] = [i]
        continue
      indexes_for_classes[value].append(i)
    return indexes_for_classes

  def predict(self, X):
    MFs_learned = []
    for i in range(len(self.F)):
      MFs_learned.append(conv.makeMFMatrix(self.F[i], self.len_f[i]))

    X_OH = createOneHot_X(X)
    _, P = conv.evaluateClassifier(X_OH, MFs_learned, self.W)
    pred = np.argmax(P, axis=0)
    pred = [x + 1 for x in pred]
    
    return pred


  def createBalancedDataset(self, X_unbalanced, y_unbalanced, index_dict, K):
    # using counter due to efficiency: https://stackoverflow.com/questions/2600191/how-can-i-count-the-occurrences-of-a-list-item, already sorted
    _, min_count =  min(Counter(y_unbalanced).items(), key=itemgetter(1)) # getting min element out of it https://stackoverflow.com/questions/53046410/min-value-from-a-counter-result-in-python

    X_balanced = np.zeros((X_unbalanced.shape[0], min_count*K)) 
    y_balanced = []
    counter = 0
    for label, indexes in index_dict.items():      # from each class indexes
      sampled = sample(indexes,min_count ) # take a random with replacement of min size
      for idx in sampled:
        X_balanced[:,counter] = X_unbalanced[:,idx]
        y_balanced.append(y_unbalanced[idx])
        counter +=1

    return X_balanced, y_balanced


time: 1.45 s


## 4 Training

## Bonus 1: Elaborating multiple convolutional layers and numbers of filters

Layers: 2 to 8

Fiters:  20, 30, 40

In [0]:
#Network definition
# network parameters for tuning
eta = 0.01
rho = 0.9
mu = 0
sigma = np.sqrt(2 / d) # he init
k=3

n_batch = 100
n_update = 500
n_epochs = 100
balanced = True

X = createOneHot_X(names_train)
Y = createOneHot_Y(labels_train)
X_val = createOneHot_X(names_val)
Y_val = createOneHot_Y(labels_val)

#stroe results
train_store=[]
val_store=[]
# layer specific parameters

# iterate over number of layers
for L in range(2, 9, 2):
  # iterate over fiter sizes
  for nfilter in range(20, 41, 10):
    ns = [nfilter]*L
    ks = [k]*L
    print("Results for:", L, nfilter)
    # building object and initializing
    conv = ConvNet( K, eta, rho, ns, ks, d, n_len, K, L)   
    conv.init( mu, sigma) #layer1
    
    train, validate, W_learned, F_learned = conv.miniBatchGD( X, Y,  X_val, Y_val, labels_train, labels_val, n_batch, n_update, n_epochs, balanced)
    train_store.append(train)
    val_store.append(validate)
    
    print(np.min(train[0]), np.max(train[1]), train[0][-1], train[1][-1])
    print(np.min(validate[0]), np.max(validate[1]), validate[0][-1], validate[1][-1])


Results for: 2 20
1.569178315680376 0.5071724416607738 1.569178315680376 0.5071724416607738
2.146757005958165 0.38492063492063494 2.14760170257085 0.38492063492063494
Results for: 2 30
1.522574562197274 0.5335387412869987 1.522574562197274 0.5335387412869987
2.1559591319426614 0.38492063492063494 2.1559591319426614 0.38492063492063494
Results for: 2 40
1.4015795636697044 0.5772805333872109 1.4015795636697044 0.5772805333872109
2.134975953703798 0.42857142857142855 2.134975953703798 0.4246031746031746
Results for: 4 20
1.3941087265263736 0.5737448227093646 1.4258030148517238 0.5660167693706435
2.2788884454857166 0.38492063492063494 2.3128844887982645 0.373015873015873
Results for: 4 30
1.4773893276187222 0.5510152540660673 1.4912012045703384 0.547934134761087
2.109145120938567 0.3888888888888889 2.109145120938567 0.3888888888888889
Results for: 4 40
1.2701610583247138 0.6263258915041924 1.2701610583247138 0.6263258915041924
2.3036750420085137 0.4087301587301587 2.4528543408109122 0.4087